In [37]:
import new_adventure as na
import new_adventure.derivative_free_estimation as dfe
import numpy as np
import matplotlib.pyplot as plt
import time

import numpy as np
import time

import multiprocessing
from os import getpid

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def f(i):
    print(i)
    return 1

def multi_second_shift_estimator_task(x_0, F, process_N, alpha, L_sample_points, L_grads, pid):
    x_0 = np.ones(dim) / np.linalg.norm(np.ones(dim))
    # Notice, we should be sharing the radius. So some lock is probably needed to synchronise 
    sample_points = dfe.hit_run(x_0, F, x_0.shape[0], process_N, alpha)
    out_grads = F.f1(sample_points)
    L_sample_points.append(sample_points)
    L_grads.append(out_grads)

def new_beta_second_shift_estimator(F, x_0, alpha, N, control_variate=True, num_processes=1):


    manager = multiprocessing.Manager()
    pool = multiprocessing.Pool(processes=num_processes)
    L_sample_points = manager.list()
    L_grads = manager.list()
    pool_workers = []
    a = time.time()
    for i in range(num_processes):
        p = pool.apply_async(multi_second_shift_estimator_task, (None, F, N // num_processes, alpha, L_sample_points, L_grads, i,))
        pool_workers.append(p)
    
    pool_workers = [p.wait() for p in pool_workers]

    sample_points = np.vstack(list(L_sample_points))
    out_grads = np.vstack(list(L_grads))

    second_shift_est = dfe.new_proper_cov(sample_points, out_grads)
    return second_shift_est.dot(np.linalg.inv(dfe.np_new_cov(sample_points)))

In [6]:
np.random.seed(10)
dim = 2500
num_barriers = dim * 2
dirs = np.random.normal(size=(num_barriers, dim)) # sample gaussian and normalize 
ws = dirs/np.linalg.norm(dirs, axis=1).reshape(-1, 1)
bs = np.ones(num_barriers)

barrier = na.Barriers.LogPolytopeBarrier(ws, bs)

F = na.Functions.Linear(np.ones(dim))

F = barrier # na.Functions.LinearCombination(F, barrier, [1, 1])

In [9]:
xs = np.ones(dim) / np.linalg.norm(np.ones(dim))

# a = time.time()
# new_beta_second_shift_estimator(F, xs, 200, 10000, control_variate=True, num_processes=1)
# print(time.time() - a)

for _ in range(100):
    a = time.time()
    print(len(new_beta_second_shift_estimator(F, xs, 200, 100, control_variate=True, num_processes=8)))
    print(time.time() - a)

2500
3.498368263244629
2500
3.569193124771118
2500
3.6273136138916016
2500
3.503836154937744
2500
3.5192155838012695
2500
3.440156936645508
2500
3.476142168045044
2500
3.5859899520874023
2500
3.638495683670044
2500
3.4816277027130127
2500
3.5911083221435547
2500
3.49714994430542
2500
3.5256919860839844
2500
3.4841487407684326
2500
3.637873649597168
2500
3.5838942527770996
2500
3.507283926010132
2500
3.5485706329345703
2500
3.632079839706421
2500
3.683032274246216
2500
3.4433555603027344
2500
3.6062872409820557
2500
3.624016761779785
2500
3.6580941677093506
2500
3.636711359024048
2500
3.555225372314453
2500
3.6881463527679443
2500
3.5865070819854736
2500
3.5497050285339355
2500
3.6343209743499756
2500
3.509509563446045
2500
3.5800256729125977
2500
3.631127119064331
2500
3.593026876449585
2500
3.5205726623535156
2500
3.487527370452881
2500
3.6842575073242188
2500
3.5844109058380127
2500
3.633708953857422
2500
3.6102678775787354
2500
3.574589252471924
2500
3.6844475269317627
2500
3.750444

In [56]:
a = time.time()
dfe.new_beta_second_shift_estimator(F, xs, 200, 10000, control_variate=True)
print(time.time() - a)

9.42070984840393


In [147]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    print(x)
    return x*x


pool = multiprocessing.Pool(4)
    

# launching multiple evaluations asynchronously *may* use more processes
multiple_results = [pool.apply_async(f, (i,)) for i in range(4)]
# print([res.get(timeout=1) for res in multiple_results])

# res = pool.apply_async(f, (20,))      # runs in *only* one process
# print(res.get(timeout=1))             # prints "400"

1
2
0
3


Process ForkPoolWorker-286:
Process ForkPoolWorker-288:
Traceback (most recent call last):


In [12]:
from multiprocessing import Process, Queue
a = time.time()
def sum(a,b):
    su=0
    for i in range(a,b):
        su+=i
    q.put(su)

q= Queue()

p1=Process(target=sum, args=(1,25*10**7))
p2=Process(target=sum, args=(25*10**7,5*10**8))
p3=Process(target=sum, args=(5*10**8,75*10**7))
p4=Process(target=sum, args=(75*10**7,10**9))

p1.start()
p2.start()
p3.start()
p4.start()

r1=q.get()
r2=q.get()
r3=q.get()
r4=q.get()

print(r1+r2+r3+r4)
print(time.time() - a)

499999999500000000
13.995182991027832


In [ ]:
def sum(a,b):
    su=0
    for i in range(a,b):
        su+=i
    print(su)
a = time.time()
sum(1,10**9)
print(time.time() - a)

In [38]:
import jax.numpy as jnp
import jax

In [43]:
xs_np = np.array([[1., 1.], [-1., 1.]])
xs = jnp.array(xs_np)
# xs[jnp.any(xs > 0, axis=1)] = jnp.inf 
jax.ops.index_update(xs, jnp.any(xs < 0, axis=1), jnp.inf)
xs 

DeviceArray([[ 1.,  1.],
             [-1.,  1.]], dtype=float32)

In [40]:
xs_np[np.any(xs_np < 0, axis=1)] = np.inf 
xs_np

array([[ 1.,  1.],
       [inf, inf]])

In [41]:
xs_np

array([[ 1.,  1.],
       [inf, inf]])

In [42]:
np.nan > 1

False